# 75 pandas Exercises: Exercises 21 to 30

Exercises 21 to 30 from [here](https://www.machinelearningplus.com/python/101-pandas-exercises-python/). Each exercise includes the question, the input and the solution's code. Sometimes, alternative solutions and comments to better explain solutions/pandas functionality are offered.

Requirements: 
+ `pandas`
+ `numpy`

Happy Pandasing! 🐼

## Imports

In [1]:
import pandas as pd
import numpy as np # required for some questions

---

## Exercises

### 🐼 Exercise 21

**How to convert a series of date-strings to a timeseries?** 

Input

In [3]:
ser = pd.Series(['01 Jan 2010', '02-02-2011', '20120303', '2013/04/04', '2014-05-05', '2015-06-06T12:20'])

To convert this to a datetime format understood by Pandas, we would need to specify a format that the parser would identify (something alone the lines of DD-MM-YYYY). However, we have several different date formats in `ser`, so we would need multiple parsing specifications. Thankfully, Pandas already has a built-in parsing engine that can automatically extract dates from common timestamp formats.  



In [4]:
new_ser = pd.to_datetime(ser, infer_datetime_format=True) # infer datetime format to activate the built-in parsing engine

In [5]:
print(new_ser)

0   2010-01-01 00:00:00
1   2011-02-02 00:00:00
2   2012-03-03 00:00:00
3   2013-04-04 00:00:00
4   2014-05-05 00:00:00
5   2015-06-06 12:20:00
dtype: datetime64[ns]


All date strings converted to a common, continuous format, that can be easily used as a `DataFrame`'s  index.   

### 🐼 Exercise 22

**Get the day of month, week number, day of year and day of week from a series of date strings.**

Input

In [11]:
ser = pd.Series(['01 Jan 2010', '02-02-2011', '20120303', '2013/04/04', '2014-05-05', '2015-06-06T12:20'])

First, and building upon exercise 22, let's convert everything to the `pd.Timestamp` format.

In [12]:
ser_ts = pd.to_datetime(ser)
print(type(ser_ts))
print(type(ser_ts[0]))

<class 'pandas.core.series.Series'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [14]:
print(ser_ts)

0   2010-01-01 00:00:00
1   2011-02-02 00:00:00
2   2012-03-03 00:00:00
3   2013-04-04 00:00:00
4   2014-05-05 00:00:00
5   2015-06-06 12:20:00
dtype: datetime64[ns]


Okay, we know have a `pd.Series` of timestamps. The `pd.Timestamp` is a complex object holding, in its attributes, all the metadata we are looking for, so let's extract it with a bit of list comprehension magic.

In [31]:
day_of_month = [ts.days_in_month for ts in ser_ts]
week_number = [ts.weekofyear for ts in ser_ts]
day_of_year = [ts.dayofyear for ts in ser_ts]
day_of_week = [ts.day_name() for ts in ser_ts] # previously, it was weekday_name, but it's being phased out in favour of the method day_name().
# It's a method because it has a locale argument (a same absolute timestamp can mean different week days, according to the timezone).

Checking:

In [32]:
print("Day in month: {}".format(day_of_month))
print("Week number: {}".format(week_number))
print("Day of the year: {}".format(day_of_year))
print("Weekday name: {}".format(day_of_week))

Day in month: [31, 28, 31, 30, 31, 30]
Week number: [53, 5, 9, 14, 19, 23]
Day of the year: [1, 33, 63, 94, 125, 157]
Weekday name: ['Friday', 'Wednesday', 'Saturday', 'Thursday', 'Monday', 'Saturday']


_Voilà!_

### 🐼 Exercise 23

**Convert year-month string to dates corresponding to the 4th day of the month?** Change `ser` to dates that start with the 4th day of the respective months.

Input

In [34]:
ser = pd.Series(['Jan 2010', 'Feb 2011', 'Mar 2012'])

Okay, so maybe we can go for a bit of timestamp arithmetic? Let's start by creating a timestamp `pd.Series`.

In [36]:
ser_ts = pd.to_datetime(ser)
print(ser_ts)

0   2010-01-01
1   2011-02-01
2   2012-03-01
dtype: datetime64[ns]


Basically, we can implement timestamp arithmetic on `pd.Timestamps` objects using `pd.Timedelta` to create our time parcels and, well... arithmetic operators. 

In [42]:
delta_time = pd.Timedelta(days=3)
ser_ts_delta = ser_ts + delta_time
ser_ts_delta.head()

0   2010-01-04
1   2011-02-04
2   2012-03-04
dtype: datetime64[ns]

_Easy peasy!_

### 🐼 Exercise 24

**Filter words that contain at least 2 vowels from a `pd.Series`?** From `ser`, extract the words containing at least 2 vowels.

Input

In [41]:
ser = pd.Series(['Apple', 'Orange', 'Plan', 'Python', 'Money'])

So, let's count the vowels in each element of the `pd.Series` and filter based on that. The best way to count elements in a list (apart from mcgyvering it) is to use `collections.Counter`. 

Let's pre-process the list: 

In [66]:
vowels = list('aeiou') # quick & dirty way to transform a word into a list of characters
ser_lower = ser.apply(lambda x: x.lower()) # putting things in lowercase

Let's map each element in `ser` to its number of vowels. Calling `Counter()` on a string returns a dictionary with all the letters in that string and the number of occurrences of that letter. By accessing only the keys of that dictionary that are vowels and summing them up, we get the vowel count for each entry in `ser_lower`. 

**Note**: The `.get()` method of a dictionary allows setting a default value in case a key is not found. This is crucial because if the vowel we are sweeping for is not present in the word, it would error out. Instead, we set it to 0, meaning no vowel is present. 

In [94]:
from collections import Counter
ser_vowels_count = ser_lower.map(lambda x: sum([Counter(x).get(v, 0) for v in vowels]))

With the vowel counts in the `pd.Series` `ser_vowels_count`, let's filter: 

In [93]:
ser[ser_vowels_count >= 2]

0     Apple
1    Orange
4     Money
dtype: object

It's done. Partly, based on the suggested solution. A little bit complex upon a first inspection because a lot is going on on that one-liner above, but hey!, learned about `collections.Counter`, a very useful tool. 

### 🐼 Exercise 25

**Filter valid emails from a `pd.Series`**. Extract the valid emails from the series emails. The regex pattern for valid emails is provided as reference.

Input & regular expression pattern to detect valid emails

In [26]:
emails = pd.Series(['buying books at amazom.com', 'rameses@egypt.com', 'matt@t.co', 'narendra@modi.com'])
pattern ='[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,4}'

First, we need to import a regular expression processor

In [1]:
import re

Now, we just test every element of the series against the regular expression and extract the email in case there's a match: 

In [18]:
is_mail = emails.map(lambda x: bool(re.match(pattern, x)))
emails_only = emails[is_mail]
print(emails_only.tolist())

['rameses@egypt.com', 'matt@t.co', 'narendra@modi.com']


This solution was shamelessly stolen from the suggested solution. I am starting to realize that applying the `map()` method and using a list comprehension are interchangeable. 

Another of the suggested solutions is using the string representation of the series, againts which the regular expression processor can run. 

In [37]:
valid_emails_alt = emails.str.findall(pattern, flags=re.IGNORECASE)[1:] # the first element is an empty, so let's ignore it
print(list(valid_emails_alt))

[['rameses@egypt.com'], ['matt@t.co'], ['narendra@modi.com']]


However, this `pd.Series` is a complex nested object with an extra level of hierarchy, which we need to manually simplify. 

In [38]:
valid_emails_alt = [email[0] for email in valid_emails_alt]
print(valid_emails_alt)

['rameses@egypt.com', 'matt@t.co', 'narendra@modi.com']


_Simplified it is._

### 🐼 Exercise 26

**Get the mean of a `pd.Series` grouped by another `pd.Series`.** Compute the mean of `weights` of each `fruit`.

Input

In [44]:
fruit = pd.Series(np.random.choice(['apple', 'banana', 'carrot'], 10))
weights = pd.Series(np.linspace(1, 10, 10))

So, let's take a look at our lists: 

In [45]:
print(fruit)

0    banana
1    carrot
2     apple
3    banana
4    carrot
5    carrot
6     apple
7    carrot
8    banana
9     apple
dtype: object


In [46]:
print(weights)

0     1.0
1     2.0
2     3.0
3     4.0
4     5.0
5     6.0
6     7.0
7     8.0
8     9.0
9    10.0
dtype: float64


So, the idea is to get all weights corresponding to a certain fruit and compute their average. Can we... `groupby` the value in `fruit`?

In [49]:
grouped = weights.groupby(fruit)
print(type(grouped))

<class 'pandas.core.groupby.groupby.SeriesGroupBy'>


In [ ]:
We now have a `pd.SeriesGroupBy` object, an agglomeration of groups upon which we can apply standard `pd.Series` operations, like getting the averages. 

In [54]:
means = grouped.mean()
print(means)

apple     6.666667
banana    4.666667
carrot    5.250000
dtype: float64


We even get a `pd.Series` back indexed by the group's name. 

In [55]:
print(means.index)

Index(['apple', 'banana', 'carrot'], dtype='object')


_Niiiice!_

### 🐼 Exercise 27

**Compute the euclidean distance between two series?** Compute the euclidean distance between the points `pd.Series` `p` and `q`.
The Euclidean distance is given by $\sqrt{\sum(p_{i} - q_{i})^{2}}$. 

Input

In [58]:
p = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
q = pd.Series([10, 9, 8, 7, 6, 5, 4, 3, 2, 1])

Let's be practical about it and praise the interoperability of `pd.Series` and `numpy`: 

In [65]:
euclidean_dist = np.sum((p-q)**2)**0.5 # For extra readability, we could go for np.sqrt(np.sum((p-q)**2))
print(euclidean_dist)

18.16590212458495


Just as expected!

### 🐼 Exercise 28

**Find all the local maxima (or peaks) in a numeric series.** Get the positions of peaks (values surrounded by smaller values on both sides) in `ser`.

Input

In [3]:
ser = pd.Series([2, 10, 3, 4, 9, 10, 2, 7, 3])

Okay, this is one exercise that doesn't really require all that much Pandasing and is easily solvable through `numpy`. We differentiate the array on both the forward and backwards direction and the indexes of the negative values in both directions will be our peaks. 

In [21]:
diff_forward = np.diff(ser)
diff_backwards = np.diff(ser[::-1])
peaks = np.intersect1d(np.where(diff_forward < 0)[0], np.where(diff_backwards < 0)[0])
print(peaks)

[1 5 7]


_A simple enough solution._

### 🐼 Exercise 29

**Replace missing spaces in a string with the least frequent character.** Replace the spaces in `my_str` with the least frequent character.

Input

In [24]:
my_str = 'dbc deb abed gade'

So, we first have to use Pandas to determine the least frequent character. Let's create a `pd.Series` with one character per row. 

In [38]:
str_series = pd.Series(data=[i for i in my_str])
print(str_series)

0     d
1     b
2     c
3      
4     d
5     e
6     b
7      
8     a
9     b
10    e
11    d
12     
13    g
14    a
15    d
16    e
dtype: object


Now, to find the least frequent value, we use `pd.Series.value_counts()`, which returns us, for each unique value in the `pd.Series`, their occurrence count - a sort of an histogram ordered, by default, in descending order. 

In [41]:
print(str_series.value_counts())

d    4
     3
b    3
e    3
a    2
g    1
c    1
dtype: int64


In [43]:
least_freq = str_series.value_counts().index[-1]

Now, let's replace all the empty spaces with this `least_freq` character (using the built-in `pd.Series.replace()` method) and convert the series back to a string.

In [47]:
replaced = str_series.replace(' ', value=least_freq).tolist()
print(''.join(replaced)) # Simple trick to convert a list of characters into a string

dbccdebcabedcgade


_Voilá!_

### 🐼 Exercise 30

**Create a `TimeSeries` starting at ‘2000-01-01’ with the following 10 weekends (saturdays), each day having as values a random number.**

So, the first of January of 2000 was actually a Saturday, so first we just have to create a wekly `TimeSeries` starting that day and lasting for 10 weeks. This TimeSeries is going to be our index (indexing random values, in this case). 

In [55]:
index_weeks = pd.date_range(start='2000-01-01', periods=10, freq = 'W-SAT')
print(index_weeks)

DatetimeIndex(['2000-01-01', '2000-01-08', '2000-01-15', '2000-01-22',
               '2000-01-29', '2000-02-05', '2000-02-12', '2000-02-19',
               '2000-02-26', '2000-03-04'],
              dtype='datetime64[ns]', freq='W-SAT')


(the `freq` argument specifies that we want a weekly frequency starting at the closest saturday. The entire syntax is described [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases))

Now, we just create a `pd.Series` with random data. A `pd.Series` indexed by `index_weeks`, of course.

In [60]:
dates_and_random = pd.Series(data=np.random.randint(low=0, high=100, size=len(index_weeks)), index=index_weeks)
print(dates_and_random)

2000-01-01    42
2000-01-08    44
2000-01-15    38
2000-01-22    15
2000-01-29    39
2000-02-05    37
2000-02-12    85
2000-02-19    23
2000-02-26    80
2000-03-04    93
Freq: W-SAT, dtype: int64


---

### See ya next notebook! 🐼

Can't stop? [Exercises 31 to 40](TODO).